In [1]:
import mysql.connector
from dbconfig import db_config

from tqdm import tqdm

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from sklearn.linear_model import Ridge, Lasso

#### Load data from SQL database

In [2]:
mydb = mysql.connector.connect(
        host = db_config["host"],
        user = db_config["user"],
        passwd = db_config["password"],
        database = db_config["database"]
    )
mycursor = mydb.cursor()

In [3]:
mycursor.execute("SELECT * FROM Stocks")
aapl = mycursor.fetchall()
sequence = mycursor.column_names

#### Data Preprocessing

In [4]:
df = pd.DataFrame(aapl, columns = sequence).set_index('date').drop(['symbol', 'high', 'low', 'open'], axis = 1)
df = df.fillna('ffill').fillna('bfill')
df['close'] = pd.to_numeric(df['close'])
df['log_return'] = np.log(df['close']) - np.log(df['close'].shift())
df.dropna(axis=0, inplace=True)
df

,close,volume,log_return
date,,,
2015-06-02,131.26,35078387,-0.015497
2015-06-03,135.94,32034911,0.035034
2015-06-04,131.78,38463484,-0.031080
2015-06-05,131.58,37314004,-0.001519
2015-06-08,133.70,53624730,0.015983
...,...,...,...
2020-05-22,330.48,20958094,0.036892
2020-05-26,329.79,32870796,-0.002090
2020-05-27,329.09,28621405,-0.002125


### Feature Engineer - Technical Indicators

#### RSI - 5 day

In [5]:
def MA(df, col, n=5):
    '''Calculate an n-period Moving Average of the specified column in a dataframe.
    Args:
        df : dataframe with specified column
        col : specified column to calculate moving average
        n : moving average period
    Returns:
        df : dataframe with moving average column
    '''
    df[str(col)+'_MA_'+str(n)] = df[col].rolling(n).mean()
    
    return df

In [6]:
def RSI(df, col='close', n=5):
    '''Calculate an n-period RSI indicator of the specified column in a dataframe.
    Args:
        df : dataframe with specified column
        col : specified column to calculate RSI
        n : RSI period
    Returns:
        df : dataframe with RSI column
    '''
    df = df.copy()
    df['return'] = df[col].diff()
    df['up'] = df['return'][df['return'] > 0]
    df['down'] = df['return'][df['return'] < 0]
    df.fillna(0, inplace=True)
    df = MA(df, 'up', n=5)
    df = MA(df, 'down', n=5)
    df['RSI_'+str(n)] = 100 - 100 / (1 + df['up_MA_5'] / df['down_MA_5'])
    df.drop(['return','up','down','up_MA_'+str(n),'down_MA_'+str(n)], axis=1, inplace=True)
    
    return df

In [7]:
RSI(df, col='close', n=5)

,close,volume,log_return,RSI_5
date,,,,
2015-06-02,131.26,35078387,-0.015497,NaN
2015-06-03,135.94,32034911,0.035034,NaN
2015-06-04,131.78,38463484,-0.031080,NaN
2015-06-05,131.58,37314004,-0.001519,NaN
2015-06-08,133.70,53624730,0.015983,278.688525
...,...,...,...,...
2020-05-22,330.48,20958094,0.036892,186.257563
2020-05-26,329.79,32870796,-0.002090,920.769231
2020-05-27,329.09,28621405,-0.002125,640.106952


#### ROC - 5 day

In [8]:
def ROC(df, col='close', n=5):
    '''Calculate an n-period ROC indicator of the specified column in a dataframe.
    Args:
        df : dataframe with specified column
        col : specified column to calculate ROC
        n : ROC period
    Returns:
        df : dataframe with ROC column
    '''
    df = df.copy()
    df['ROC_'+str(n)] = round(((df[col] - df[col].shift(n)) / df[col].shift(n)) * 100, 4)
    
    return df

In [9]:
ROC(df, col='close', n=5)

,close,volume,log_return,ROC_5
date,,,,
2015-06-02,131.26,35078387,-0.015497,NaN
2015-06-03,135.94,32034911,0.035034,NaN
2015-06-04,131.78,38463484,-0.031080,NaN
2015-06-05,131.58,37314004,-0.001519,NaN
2015-06-08,133.70,53624730,0.015983,NaN
...,...,...,...,...
2020-05-22,330.48,20958094,0.036892,3.6280
2020-05-26,329.79,32870796,-0.002090,0.3958
2020-05-27,329.09,28621405,-0.002125,0.5715


#### STOCH - 5 %k days, 3 %d days

In [10]:
def Stoch_Osc(df, col='close', k=5, d=3):
    '''Calculate a k-period slow Stochastic Oscillator indicator and a d-period fast Stochastic Oscillator indicator 
    of the specified column in a dataframe.
    Args:
        df : dataframe with specified column
        col : specified column to calculate Stochastic Oscillator
        k : slow Stochastic Oscillator period
        d : fast Stochastic Oscillator period
    Returns:
        df : dataframe with slow (%k) and fast (%d) Stochastic Oscillator columns
    '''
    df = df.copy()
    df['stoch_%k'] = (df[col] - df[col].rolling(k).min()) / (df[col].rolling(k).max() - df[col].rolling(k).min())
    df = MA(df, 'stoch_%k', n=d)
    df.rename(columns={'stoch_%k_MA_'+str(d) : 'stoch_%d_'+str(d)}, errors="raise", inplace=True)
    
    return df

In [11]:
Stoch_Osc(df, col='close', k=5, d=3)

,close,volume,log_return,stoch_%k,stoch_%d_3
date,,,,,
2015-06-02,131.26,35078387,-0.015497,NaN,NaN
2015-06-03,135.94,32034911,0.035034,NaN,NaN
2015-06-04,131.78,38463484,-0.031080,NaN,NaN
2015-06-05,131.58,37314004,-0.001519,NaN,NaN
2015-06-08,133.70,53624730,0.015983,0.521368,NaN
...,...,...,...,...,...
2020-05-22,330.48,20958094,0.036892,1.000000,0.530908
2020-05-26,329.79,32870796,-0.002090,0.942356,0.647452
2020-05-27,329.09,28621405,-0.002125,0.883876,0.942077


#### MACD - 5/10 short days, 20/40 long days, 5 signal days

In [12]:
def MACD(df, col='close', long=20, short=5, signal=5):
    '''Calculate the distance between a MACD indicator and its signal line of the specified column in a dataframe.
    Args:
        df : dataframe with specified column
        col : specified column to calculate MACD and its signal line
        short : short MACD period 
        long : long MACD period
        signal : signal line period    
    Returns:
        df : dataframe with the distance between MACD and signal line column
    '''
    df = df.copy()
    df = MA(df, col='close', n=short)
    df = MA(df, col='close', n=long)
    df['MACD'] = df[str(col)+'_MA_'+str(long)] - df[str(col)+'_MA_'+str(short)]
    df = MA(df, col='MACD', n=signal)
    df['MACD_diff_'+str(long)+'-'+str(short)+'-'+str(signal)] = df['MACD'] - df['MACD_MA_'+str(signal)]
    df.drop([str(col)+'_MA_'+str(long), str(col)+'_MA_'+str(short), 'MACD', 'MACD_MA_'+str(signal)], axis=1, inplace=True)
    
    return df

In [13]:
MACD(df, col='close', short=5, long=20, signal=5)

,close,volume,log_return,MACD_diff_20-5-5
date,,,,
2015-06-02,131.26,35078387,-0.015497,NaN
2015-06-03,135.94,32034911,0.035034,NaN
2015-06-04,131.78,38463484,-0.031080,NaN
2015-06-05,131.58,37314004,-0.001519,NaN
2015-06-08,133.70,53624730,0.015983,NaN
...,...,...,...,...
2020-05-22,330.48,20958094,0.036892,-0.3571
2020-05-26,329.79,32870796,-0.002090,1.1180
2020-05-27,329.09,28621405,-0.002125,2.3456


In [14]:
MACD(df, col='close', short=10, long=40, signal=5)

,close,volume,log_return,MACD_diff_40-10-5
date,,,,
2015-06-02,131.26,35078387,-0.015497,NaN
2015-06-03,135.94,32034911,0.035034,NaN
2015-06-04,131.78,38463484,-0.031080,NaN
2015-06-05,131.58,37314004,-0.001519,NaN
2015-06-08,133.70,53624730,0.015983,NaN
...,...,...,...,...
2020-05-22,330.48,20958094,0.036892,1.30270
2020-05-26,329.79,32870796,-0.002090,2.27805
2020-05-27,329.09,28621405,-0.002125,1.79010


#### MOM - 5/20 days

In [15]:
def Momentum(df, col='close', n=5):
    '''Calculate an n-period Momentum indicator of the specified column in a dataframe.
    Args:
        df : dataframe with specified column
        col : specified column to calculate Momentum
        n : Momentum period    
    Returns:
        df : dataframe with Momentum column
    '''
    df = df.copy()
    df['momentum_'+str(n)] = df[col] - df[col].shift(n)
    
    return df

In [16]:
Momentum(df, col='close', n=5)

,close,volume,log_return,momentum_5
date,,,,
2015-06-02,131.26,35078387,-0.015497,NaN
2015-06-03,135.94,32034911,0.035034,NaN
2015-06-04,131.78,38463484,-0.031080,NaN
2015-06-05,131.58,37314004,-0.001519,NaN
2015-06-08,133.70,53624730,0.015983,NaN
...,...,...,...,...
2020-05-22,330.48,20958094,0.036892,11.57
2020-05-26,329.79,32870796,-0.002090,1.30
2020-05-27,329.09,28621405,-0.002125,1.87


In [17]:
Momentum(df, col='close', n=20)

,close,volume,log_return,momentum_20
date,,,,
2015-06-02,131.26,35078387,-0.015497,NaN
2015-06-03,135.94,32034911,0.035034,NaN
2015-06-04,131.78,38463484,-0.031080,NaN
2015-06-05,131.58,37314004,-0.001519,NaN
2015-06-08,133.70,53624730,0.015983,NaN
...,...,...,...,...
2020-05-22,330.48,20958094,0.036892,42.24
2020-05-26,329.79,32870796,-0.002090,38.74
2020-05-27,329.09,28621405,-0.002125,42.14


#### VOSC - 2, 5 days

In [18]:
def Vol_Osc(df, col='volume', slow=5, fast=2):
    '''Calculate an n-period Volume Oscillator indicator of the specified column in a dataframe.
    Args:
        df : dataframe with specified column
        col : specified column to calculate Volume Oscillator
        n : Volume Oscillator period
    Returns:
        df : dataframe with Volume Oscillator column
    '''
    df = df.copy()
    df = MA(df, col=col, n=slow)
    df = MA(df, col=col, n=fast)
    df['vol_'+str(slow)+'-'+str(fast)] = df[str(col)+'_MA_'+str(slow)] - df[str(col)+'_MA_'+str(fast)]
    df.drop([str(col)+'_MA_'+str(slow), str(col)+'_MA_'+str(fast)], axis=1, inplace=True)
    
    return df

In [19]:
Vol_Osc(df, col='volume', slow=5, fast=2)

,close,volume,log_return,vol_5-2
date,,,,
2015-06-02,131.26,35078387,-0.015497,NaN
2015-06-03,135.94,32034911,0.035034,NaN
2015-06-04,131.78,38463484,-0.031080,NaN
2015-06-05,131.58,37314004,-0.001519,NaN
2015-06-08,133.70,53624730,0.015983,-6166263.8
...,...,...,...,...
2020-05-22,330.48,20958094,0.036892,3808134.0
2020-05-26,329.79,32870796,-0.002090,116661.8
2020-05-27,329.09,28621405,-0.002125,-3249324.3


#### All Technical Indicators

In [20]:
def all_ind(df):
    
    df = df[['close', 'volume', 'log_return']]

    indicator_lst = [ROC(df, col='close', n=5), Stoch_Osc(df, col='close', k=5, d=3), 
                     MACD(df, col='close', short=5, long=20, signal=5), Momentum(df, col='close', n=5), 
                     Vol_Osc(df, col='volume', slow=5, fast=2)]

    df_ind = RSI(df, col='close', n=5)
    for ind in indicator_lst:
        df_ind = pd.merge(df_ind, ind, on = ['date', 'close', 'volume', 'log_return'])
    
    return df_ind

In [21]:
df_ind = all_ind(df)
df_ind.dropna(axis=0, inplace=True)
df_ind

,close,volume,log_return,RSI_5,ROC_5,stoch_%k,stoch_%d_3,MACD_diff_20-5-5,momentum_5,vol_5-2
date,,,,,,,,,,
2015-07-06,130.00,28685277,0.000693,247.126437,1.3566,0.614035,0.732164,-0.4058,1.74,7884185.2
2015-07-07,128.04,48877116,-0.015192,-186.075949,-1.2189,0.000000,0.398830,-0.2220,-1.58,-2444787.5
2015-07-08,123.27,61522913,-0.037966,-59.036145,-3.8830,0.000000,0.204678,0.3083,-4.98,-15455041.1
2015-07-09,126.01,80736031,0.021984,-55.599214,-3.8825,0.407132,0.135711,0.9142,-5.09,-21455456.4
2015-07-10,128.59,63488995,0.020268,-409.848485,-1.0161,0.790490,0.399208,0.6740,-1.32,-15450446.6
...,...,...,...,...,...,...,...,...,...,...
2020-05-22,330.48,20958094,0.036892,186.257563,3.6280,1.000000,0.530908,-0.3571,11.57,3808134.0
2020-05-26,329.79,32870796,-0.002090,920.769231,0.3958,0.942356,0.647452,1.1180,1.30,116661.8
2020-05-27,329.09,28621405,-0.002125,640.106952,0.5715,0.883876,0.942077,2.3456,1.87,-3249324.3


### Supervised Learning - Ridge Regularized Regression

In [22]:
def pred(X, Y, alpha, M):
    
    pred = pd.DataFrame(index=X.index, columns=['Yhat'])
    X = np.asarray(X)

    for i in range(M+1,len(Y)):
        Y1 = Y[1:i]
        X1 = X[0:i-1]
        ridge = Ridge(alpha=alpha, normalize=True)
        ridge.fit(X1, Y1)
        pred.iloc[i,0] = ridge.predict(X[i-1:i])[0]
    
    return pred

In [23]:
yhat = pred(df_ind.loc[:, ~df_ind.columns.isin(['close', 'volume', 'return', 'log_return'])], df_ind['log_return'], 0.5, 20)
yhat

,Yhat
date,
2015-07-06,NaN
2015-07-07,NaN
2015-07-08,NaN
2015-07-09,NaN
2015-07-10,NaN
...,...
2020-05-22,0.00653639
2020-05-26,-0.00666107
2020-05-27,-0.00438112


#### Performance Evaluation

In [24]:
def oos_eval(Y, Pred_Mean, Pred_Variance, gam=2, M=20):
    '''This function computes OOS R^2 and CEV based on a vector of predicted mean and variance
    Args:
        Y : The excess returns
        Pred_Mean : The time series of predicted mean
    Returns:
        OOS_R2 : Out-of-sample R^2
        CE : certainty equivalence of the out-of-sample return
    '''
    Pred_Mean = np.asarray(Pred_Mean)
    
    # recursively expanding window, shift to compare with next period for forecast error
    Hist_Mean = np.asarray(Y.expanding().mean().shift())
    Hist_Variance = np.asarray(Y.expanding().var().shift())
    
    OOS_SSE_Hist = np.sum((Y[M+1:]-Hist_Mean[M+1:])**2)
    OOS_SSE = np.sum((Y[M+1:]-Pred_Mean[M+1:])**2)
    OOS_R2 = 1-OOS_SSE/OOS_SSE_Hist
    
    w0 = ((1/gam)*(Hist_Mean/Hist_Variance)).clip(None,1.5)
    r0 = Y*w0
    CE_Hist = np.mean(r0[M+1:])-gam/2*np.var(r0[M+1:],ddof=1)
    
    w1 = ((1/gam)*(Pred_Mean/Hist_Variance)).clip(None,1.5)
    r1 = Y*w1
    CE = np.mean(r1[M+1:])-gam/2*np.var(r1[M+1:],ddof=1)
    
    chg_CE = CE - CE_Hist
    
    return (OOS_R2, chg_CE)

In [25]:
oos_eval(df_ind['log_return'], yhat['Yhat'], 2, M=20)

/home/jasonho0810/.local/lib/python3.8/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in minimum
  return ufunc(*args, out=out, **kwargs)


(0.07203310313604783, 0.015501671615874658)

In [26]:
np.seterr(invalid='ignore')
alphas = np.linspace(0, 1, 6)

metrics = pd.DataFrame(columns = ['R2_OOS', 'chg_CEV'], index = alphas)
for a in tqdm(alphas):
    yhat = pred(df_ind.loc[:, ~df_ind.columns.isin(['close', 'volume', 'return', 'log_return'])], df_ind['log_return'], a, 20)
    metrics.loc[a,:] = oos_eval(df_ind['log_return'], yhat['Yhat'], 2)

metrics.sort_values(by=['R2_OOS', 'chg_CEV'], ascending=False)

100%|██████████| 6/6 [00:05<00:00,  1.05it/s]


,R2_OOS,chg_CEV
0.2,0.074812,0.0164198
0.4,0.0732577,0.0158327
0.6,0.0707538,0.0151799
0.8,0.068234,0.0146301
1.0,0.0658779,0.0141484
0.0,0.0497195,0.0127425


#### Forecast - 5/10 days ahead

In [27]:
def forecast(X, Y, alpha, h=5):
    
    pred = pd.DataFrame(index=X.index, columns=['Yhat'])
    X = np.asarray(X)

    for i in range(M+1,len(Y)):
        Y1 = Y[1:i]
        X1 = X[0:i-1]
        ridge = Ridge(alpha=alpha, normalize=True)
        ridge.fit(X1, Y1)
        pred.iloc[i,0] = ridge.predict(X[i-1:i])[0]
    
    return pred

In [28]:
def forecast(df_ind, alpha, horizon=5):

    forecast = df_ind.copy()

    for i in range(horizon):

        hdate = df.index[-1] + timedelta(days=i+1)

        X1 = np.asarray(forecast.dropna().iloc[:-1,3:])
        Y1 = forecast.dropna().iloc[1:,2]
        ridge1 = Ridge(alpha=alpha, normalize=True)
        ridge1.fit(X1, Y1)

        Yhat = ridge1.predict(np.asarray(X1[-1]).reshape(1, -1))[0]
        forecast.loc[hdate, 'log_return'] = Yhat
        forecast.loc[hdate, 'close'] = np.exp(Yhat + np.log(df.loc[df.index[-1], 'close']))
        forecast.loc[hdate, 'volume'] = forecast.loc[forecast.index[-20]:forecast.index[-1], 'volume'].mean()

        forecast = all_ind(forecast)

    return forecast

In [29]:
df_forecast = forecast(df_ind, alpha=0.2, horizon=20)
df_forecast

,close,volume,log_return,RSI_5,ROC_5,stoch_%k,stoch_%d_3,MACD_diff_20-5-5,momentum_5,vol_5-2
date,,,,,,,,,,
2015-07-06,130.000000,2.868528e+07,0.000693,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-07,128.040000,4.887712e+07,-0.015192,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-08,123.270000,6.152291e+07,-0.037966,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-09,126.010000,8.073603e+07,0.021984,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-10,128.590000,6.348900e+07,0.020268,-377.304965,NaN,0.790490,NaN,NaN,NaN,-1.545045e+07
...,...,...,...,...,...,...,...,...,...,...
2020-06-14,324.982093,3.353178e+07,-0.007535,-157.033406,-1.1884,0.011207,0.493801,0.886272,-3.908460,-5.264676e+05
2020-06-15,328.468084,3.356657e+07,0.003135,270.722345,1.0941,0.574412,0.351939,0.065264,3.554778,-5.603165e+05
2020-06-16,330.882864,3.382684e+07,0.010460,299.870758,0.9262,0.972313,0.519311,-0.456121,3.036363,-4.187898e+05


In [30]:
df_forecast.loc[df_forecast.index[-20]:,'log_return'].cumsum()[-1]

0.03454208417215584